### Multi-Agent Orchestration

In [1]:
from agents import Agent

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions.",
    instructions="You provide help with math problems. Explain your reasoning and include examples."
)

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for history questions.",
    instructions="You provide help with history questions. Explain your reasoning and include examples."
)

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's question.",
    handoffs=[history_tutor_agent, math_tutor_agent]
)

In [2]:
from agents import Runner

async def main():
    result = await Runner.run(triage_agent, "What is the capital of France?")

    print(result.final_output)

In [ ]:
import asyncio
asyncio.run(main())

The capital of France is Paris. Paris has been the political, economic, and cultural center of France for centuries. Known for its art, fashion, architecture, and gastronomy, Paris is often referred to as "The City of Light" because of its role during the Age of Enlightenment and its early adoption of street lighting. Key landmarks include the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum.


### Adding Guardrails

In [3]:
from agents import GuardrailFunctionOutput, Agent, Runner
from pydantic import BaseModel

class HomeWorkOutput(BaseModel):
    is_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail Agent",
    instructions="Check if the user is asking a homework question.",
    output_type=HomeWorkOutput
)

async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeWorkOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework
    )

## All together

In [4]:
from agents import InputGuardrail

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question.",
    handoffs=[history_tutor_agent, math_tutor_agent],
    input_guardrails=[InputGuardrail(guardrail_function=homework_guardrail)]
)

async def main():
    result = await Runner.run(triage_agent, "Who was the first president of India?")
    print(result.final_output)

    result = await Runner.run(triage_agent, "What is happiness?")
    print(result.final_output)

    result = await Runner.run(triage_agent, "What is your purpose?")
    print(result.final_output)

In [7]:
import asyncio
import nest_asyncio

# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

# Now we can safely use asyncio.run
asyncio.run(main())

d:\code_2024\openai-agents-sdk-exploration\.venv\lib\site-packages\anyio\_backends\_asyncio.py:1770: RuntimeWarning: coroutine 'main' was never awaited
  async def wait(self) -> None:


The first president of India was Dr. Rajendra Prasad. He served as president from 1950 to 1962. Dr. Prasad was a key figure in the Indian independence movement and was closely associated with Mahatma Gandhi. His leadership during the drafting of the Indian Constitution played a significant role in shaping the newly independent nation. As the first president, he set many precedents for the role and is remembered for his humility and dedication to public service.
Happiness is often defined as a state of well-being and contentment characterized by feelings of joy, satisfaction, and fulfillment. It can vary greatly from person to person and is influenced by factors such as relationships, achievements, purpose, and personal values. Philosophers, psychologists, and scientists study happiness to understand its nature and how it can be achieved or increased in both individual and societal contexts.


InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire